In [3]:
SITE_DIR = r"C:\Users\benny\OneDrive\Documents\Github\site\temp3"   # where your .qmd are
ASSETS_DIR = os.path.join(SITE_DIR, "assets")
MANIFEST_PATH = os.path.join(SITE_DIR, "_asset_manifest.txt")

BASE_SITE_URL = "https://benny.istan.to"  # for /s/... links if any
DELAY_SECONDS = 1.0
OVERWRITE_EXISTING = False  # if True, redownload even if file exists


In [4]:
import os, re, csv, time, html, mimetypes
from urllib.parse import urlparse, unquote, parse_qs, urljoin

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    )
}

# Domains that usually host actual files
ASSET_HOSTS = {
    "images.squarespace-cdn.com",
    "static1.squarespace.com",
    "images.squarespace.com",
    "static.squarespace.com",
}

# Your site domain CAN be kept, but only for /s/ file URLs
SITE_HOST_ALLOW = {"benny.istan.to", "bennyistanto.squarespace.com"}

DOWNLOADABLE_EXTS = {
    ".jpg", ".jpeg", ".png", ".gif", ".webp", ".svg",
    ".pdf", ".zip", ".mp4", ".mov", ".mp3", ".wav",
    ".doc", ".docx", ".ppt", ".pptx", ".xls", ".xlsx",
}

FENCED_CODE_RE = re.compile(r"```.*?```", re.DOTALL)
MD_LINK_RE = re.compile(r"!\[[^\]]*\]\(([^)]+)\)|\[[^\]]*\]\(([^)]+)\)")
HTML_ATTR_RE = re.compile(r"""(?:src|href|data-src|data-image|data-original|poster)\s*=\s*["']([^"']+)["']""", re.IGNORECASE)
SRCSET_RE = re.compile(r"""srcset\s*=\s*["']([^"']+)["']""", re.IGNORECASE)
CSS_URL_RE = re.compile(r"""url\(\s*['"]?([^'")]+)['"]?\s*\)""", re.IGNORECASE)
RAW_URL_RE = re.compile(r"""https?://[^\s"'<>()]+""", re.IGNORECASE)

ASSET_ID_RE = re.compile(r"(\d{13}-[A-Z0-9]{8,})")


def create_session():
    s = requests.Session()
    s.headers.update(HEADERS)
    retry = Retry(
        total=6,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"],
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s


def strip_trailing_punct(u: str) -> str:
    return u.rstrip(').,;:\'"!?]>')

def normalize_url(u: str, base_site_url: str) -> str:
    u = (u or "").strip().strip("<>")
    u = html.unescape(u)
    u = strip_trailing_punct(u)
    if u.startswith("//"):
        u = "https:" + u
    if u.startswith("/"):
        u = urljoin(base_site_url.rstrip("/") + "/", u.lstrip("/"))
    return u


def looks_like_downloadable_file(u: str) -> bool:
    """
    True if URL is likely a file asset (not a page).
    """
    try:
        p = urlparse(u)
        host = p.netloc.lower()
        path = p.path or ""
        ext = os.path.splitext(path)[1].lower()

        # 1) Asset CDN hosts: require a file extension OR content-type query
        if host in ASSET_HOSTS:
            if ext in DOWNLOADABLE_EXTS:
                return True
            q = parse_qs(p.query)
            if "content-type" in q:
                return True
            return False

        # 2) Your own domain: only download if it's a Squarespace file link (/s/...) AND looks like a file
        if host in SITE_HOST_ALLOW:
            if path.startswith("/s/") and (ext in DOWNLOADABLE_EXTS or ext):
                return True
            return False

        return False
    except Exception:
        return False


def asset_key(u: str) -> str:
    p = urlparse(u)
    return f"{p.netloc.lower()}{p.path}"


def best_quality_score(u: str, width_hint: int = 0) -> int:
    score = width_hint or 0
    try:
        q = parse_qs(urlparse(u).query)
        fmt = (q.get("format", [""])[0] or "").lower()
        m = re.search(r"(\d{3,5})w", fmt)
        if m:
            score = max(score, int(m.group(1)))
        if fmt in {"original", "raw"}:
            score = max(score, 99999)
    except Exception:
        pass
    return score


def safe_filename(name: str) -> str:
    name = unquote(name).replace(" ", "_")
    name = re.sub(r"[^A-Za-z0-9._-]+", "", name)
    name = re.sub(r"_{2,}", "_", name).strip("._-")
    return name or "asset"


def fix_extension(ext: str) -> str:
    if not ext:
        return ".jpg"
    ext = ext.lower()
    if ext in [".jpe", ".jpeg"]:
        return ".jpg"
    return ext


def choose_base_name(final_url: str) -> str:
    p = urlparse(final_url)
    m = ASSET_ID_RE.search(p.path or "")
    if m:
        return m.group(1)
    base = os.path.basename(p.path) or "asset"
    stem = os.path.splitext(base)[0] or "asset"
    return safe_filename(stem)


def extract_urls_from_text(text: str, base_site_url: str):
    text = FENCED_CODE_RE.sub("", text or "")
    found = []

    # Markdown links/images
    for m in MD_LINK_RE.findall(text):
        for raw in m:
            if raw:
                raw = raw.strip().strip('"').strip("'")
                norm = normalize_url(raw, base_site_url)
                found.append((raw, norm, 0))

    # HTML attributes
    for raw in HTML_ATTR_RE.findall(text):
        raw = raw.strip()
        norm = normalize_url(raw, base_site_url)
        found.append((raw, norm, 0))

    # srcset
    for srcset in SRCSET_RE.findall(text):
        parts = [p.strip() for p in srcset.split(",") if p.strip()]
        for part in parts:
            tokens = part.split()
            raw = tokens[0].strip()
            width_hint = 0
            if len(tokens) > 1:
                m = re.match(r"(\d{2,5})w", tokens[1].lower())
                if m:
                    width_hint = int(m.group(1))
            norm = normalize_url(raw, base_site_url)
            found.append((raw, norm, width_hint))

    # CSS url(...)
    for raw in CSS_URL_RE.findall(text):
        raw = raw.strip()
        norm = normalize_url(raw, base_site_url)
        found.append((raw, norm, 0))

    # Raw URLs
    for raw in RAW_URL_RE.findall(text):
        raw = raw.strip()
        norm = normalize_url(raw, base_site_url)
        found.append((raw, norm, 0))

    # Keep only downloadables
    out = []
    for raw, norm, w in found:
        if norm and looks_like_downloadable_file(norm):
            out.append((raw, norm, w))
    return out


def collect_qmd_files(site_dir: str, assets_dir: str):
    qmds = []
    for root, _, files in os.walk(site_dir):
        # skip the assets dir itself
        if os.path.abspath(root).startswith(os.path.abspath(assets_dir)):
            continue
        for fn in files:
            if fn.endswith(".qmd"):
                qmds.append(os.path.join(root, fn))
    qmds.sort()
    return qmds


def download_one(session, url, assets_path, delay_seconds: float, overwrite: bool):
    try:
        r = session.get(url, stream=True, allow_redirects=True, timeout=45)
        if r.status_code != 200:
            return False, None, r.url, f"HTTP {r.status_code}"

        ctype = (r.headers.get("content-type") or "").split(";")[0].strip().lower()
        if ctype.startswith("text/html"):
            return False, None, r.url, "Got HTML (page/blocked/redirected)"

        ext = os.path.splitext(urlparse(r.url).path)[1]
        if not ext:
            ext = mimetypes.guess_extension(ctype) or ""
        ext = fix_extension(ext)

        base = choose_base_name(r.url)
        filename_asset = f"{base}{ext}"
        save_path = os.path.join(assets_path, filename_asset)

        if os.path.exists(save_path) and not overwrite:
            return True, filename_asset, r.url, None

        uniq = 1
        final_name = filename_asset
        while os.path.exists(save_path) and overwrite is False:
            final_name = f"{base}_{uniq}{ext}"
            save_path = os.path.join(assets_path, final_name)
            uniq += 1

        with open(save_path, "wb") as f_out:
            for chunk in r.iter_content(chunk_size=1024 * 64):
                if chunk:
                    f_out.write(chunk)

        time.sleep(delay_seconds)
        return True, final_name, r.url, None

    except Exception as e:
        return False, None, url, str(e)


def run_download(site_dir, assets_dir, manifest_path, base_site_url, delay_seconds=1.0, overwrite=False):
    print(f"--- STARTING SCAN in: {site_dir} ---")
    os.makedirs(assets_dir, exist_ok=True)

    session = create_session()
    qmd_files = collect_qmd_files(site_dir, assets_dir)
    print(f"Found {len(qmd_files)} .qmd files.")

    # key -> (best_url, score)
    best = {}

    # QMD scan
    qmd_count = 0
    for fp in qmd_files:
        with open(fp, "r", encoding="utf-8") as f:
            text = f.read()
        triples = extract_urls_from_text(text, base_site_url)
        for _raw, norm, w in triples:
            k = asset_key(norm)
            score = best_quality_score(norm, w)
            prev = best.get(k)
            if prev is None or score > prev[1]:
                best[k] = (norm, score)
        qmd_count += len(triples)

    print(f"QMD scan: {len(best)} unique downloadable assets (from {qmd_count} URL hits).")

    # Manifest scan
    if manifest_path and os.path.exists(manifest_path):
        before = len(best)
        with open(manifest_path, "r", encoding="utf-8") as f:
            for line in f:
                u = normalize_url(line.strip(), base_site_url)
                if u and looks_like_downloadable_file(u):
                    k = asset_key(u)
                    score = best_quality_score(u, 0)
                    prev = best.get(k)
                    if prev is None or score > prev[1]:
                        best[k] = (u, score)
        after = len(best)
        print(f"Manifest added: {before} -> {after} unique downloadable assets.")
    else:
        print("Manifest not found / not used (this is why you only saw ~1013 previously).")

    keys = list(best.keys())
    print(f"Downloading {len(keys)} unique assets...")

    success_csv = os.path.join(assets_dir, "_download_success.csv")
    failed_txt = os.path.join(assets_dir, "_download_failed.txt")
    url_map = {}
    success_rows = []
    failed = []

    for i, k in enumerate(keys, 1):
        url = best[k][0]
        ok, local_name, final_url, err = download_one(session, url, assets_dir, delay_seconds, overwrite)
        if ok:
            url_map[k] = local_name
            success_rows.append([k, url, final_url, local_name])
            if i <= 20 or i % 200 == 0:
                print(f"[{i}/{len(keys)}] OK  -> {local_name}")
        else:
            failed.append(f"{url}\t{err}")
            if i <= 20 or i % 200 == 0:
                print(f"[{i}/{len(keys)}] FAIL -> {url} ({err})")

    with open(success_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["asset_key", "requested_url", "final_url", "local_filename"])
        w.writerows(success_rows)

    if failed:
        with open(failed_txt, "w", encoding="utf-8") as f:
            f.write("\n".join(failed) + "\n")

    # Rewrite QMD links
    changed_files = 0
    for fp in qmd_files:
        with open(fp, "r", encoding="utf-8") as f:
            text = f.read()

        triples = extract_urls_from_text(text, base_site_url)
        if not triples:
            continue

        qmd_dir = os.path.dirname(fp)
        rel_assets = os.path.relpath(assets_dir, qmd_dir).replace("\\", "/")

        new_text = text
        changed = False
        for raw, norm, _w in triples:
            k = asset_key(norm)
            if k in url_map:
                local_rel = f"{rel_assets}/{url_map[k]}".replace("\\", "/")
                if raw in new_text:
                    new_text = new_text.replace(raw, local_rel)
                    changed = True

        if changed and new_text != text:
            with open(fp, "w", encoding="utf-8", newline="\n") as f:
                f.write(new_text)
            changed_files += 1

    print("\nDone.")
    print(f"- Downloaded: {len(url_map)}")
    print(f"- Failed: {len(failed)} (see {failed_txt})" if failed else "- Failed: 0")
    print(f"- Updated QMD files: {changed_files}")
    print(f"- Logs: {success_csv}")


In [5]:
run_download(
    site_dir=SITE_DIR,
    assets_dir=ASSETS_DIR,
    manifest_path=MANIFEST_PATH,
    base_site_url=BASE_SITE_URL,
    delay_seconds=DELAY_SECONDS,
    overwrite=OVERWRITE_EXISTING
)


--- STARTING SCAN in: C:\Users\benny\OneDrive\Documents\Github\site\temp3 ---
Found 139 .qmd files.
QMD scan: 0 unique downloadable assets (from 0 URL hits).
Manifest added: 0 -> 951 unique downloadable assets.
[1/951] OK  -> CV_ISTANTO_Benny_20250516.pdf
[2/951] OK  -> 15325333_10211141699402159_1841558926548787098_o_10211141699402159.jpg
[3/951] OK  -> mmr_pheno_eos1_2022.png
[4/951] OK  -> Screenshot2023-08-12141329.png
[5/951] OK  -> ScreenShot2022-03-19at9.42.39PM.png
[6/951] OK  -> ukr_pop_landscan_population_2021_wbg_A4L.png
[7/951] OK  -> ScreenShot2022-10-18at5.00.48PM.png
[8/951] OK  -> ScreenShot2022-10-18at4.36.45PM.png
[9/951] OK  -> 1692902253633.jpg
[10/951] OK  -> Screenshot2023-06-15075323.png
[11/951] OK  -> Heatwave_CA_2016a.png
[12/951] OK  -> IMG_2664.jpg
[13/951] OK  -> idn_annualrain_2021.png
[14/951] OK  -> viz4.png
[15/951] OK  -> Screenshot2023-05-09224753.png
[16/951] OK  -> 1589366320149-LLK82OODORI0BZMMZ0LA.png
[17/951] OK  -> 1589366433048-6WBV4YK5RSOZ65D1